# Digit Classification with Neural Networks 
## Code by Todd Holloway

Interest in neural networks, and in particular those with architechures that support deep learning, has been surging in recent years.  

In this notebook we will be revisiting the problem of digit classification on the MNIST data.  In doing so, we will introduce a new Python library, Theano, for working with neural networks.  Theano is a popular choice for neural networks as the same code can be run on either CPUs or GPUs.  GPUs greatly speed up the training and prediction, and is readily available. Amazon even offers GPU machines on EC2.  

In part 1, we'll introduce Theano, and refresh ourselves on the MNIST dataset.  In part 2, we'll create a multi-layer neural network with a simple architechure, and train it using backpropagation.  Part 3 will introduce the convolutional architechure, which can be said to be doing 'deep learning' (also called feature learning or representation learning).

#### Part 1: Basics

Lets start to look at Theano.  If later you'd like to go deeper into Theano, you may want to read this paper: http://www.iro.umontreal.ca/~lisa/pointeurs/theano_scipy2010.pdf

Install Theano if you haven't already.  Then let's load it, and set it to work with a CPU.  For reference, here is the Theano documentation: http://www.deeplearning.net/software/theano/library/

In [1]:
%matplotlib inline

import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import time

from keras.models import Sequential 
from keras.layers import Dense, Activation
from keras import optimizers

import theano 
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
print (theano.config.device) # We're using CPUs (for now)
print (theano.config.floatX) # Should be 64 bit for CPUs

np.random.seed(0)

Using TensorFlow backend.


cpu
float64


Now for MNIST data...

In [2]:
# Repeating steps from Project 1 to prepare mnist dataset. 
mnist = fetch_mldata('MNIST original', data_home='~/datasets/mnist')
X, Y = mnist.data, mnist.target
X = X / 255.0
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]
numExamples = 2000
test_data, test_labels = X[70000-numExamples:], Y[70000-numExamples:]
train_data, train_labels = X[:numExamples], Y[:numExamples]
numFeatures = train_data[1].size
numTrainExamples = train_data.shape[0]
numTestExamples = test_data.shape[0]
print ('Features = %d' %(numFeatures))
print ('Train set = %d' %(numTrainExamples))
print ('Test set = %d' %(numTestExamples))

Features = 784
Train set = 2000
Test set = 2000


Looking ahead to working with neural networks, let's prepare one additional variation of the label data.  Let's make these labels, rather than each being an integer value from 0-9, be a set of 10 binary values, one for each class.  This is sometimes called a 1-of-n encoding, and it makes working with Neural Networks easier, as there will be one output node for each class.

In [3]:
def binarizeY(data):
    binarized_data = np.zeros((data.size,10))
    for j in range(0,data.size):
        feature = data[j:j+1]
        i = feature.astype(np.int64) 
        binarized_data[j,i]=1
    return binarized_data
train_labels_b = binarizeY(train_labels)
test_labels_b = binarizeY(test_labels)
numClasses = train_labels_b[1].size
print ('Classes = %d' %(numClasses))

Classes = 10


Lets start with a KNN model to establish a baseline accuracy.

Exercise: You've seen a number of different classification algorithms (e.g. naive bayes, decision trees, random forests, logistic regression) at this point.  How does KNN scalability and accuracy with respect to the size of the training dataset compare to those other algorithms?  

In [4]:
neighbors = 1
knn = KNeighborsClassifier(neighbors)
# we'll be waiting quite a while if we use 60K examples, so let's cut it down.  You may want to run the full 60K on your own later to see what the accuracy is.
mini_train_data, mini_train_labels = X[:numExamples], Y[:numExamples] 
start_time = time.time()
knn.fit(mini_train_data, mini_train_labels)
print ('Train time = %.2f' %(time.time() - start_time))
start_time = time.time()
accuracy = knn.score(test_data, test_labels)
print ('Accuracy = %.4f' %(accuracy))
print ('Prediction time = %.2f' %(time.time() - start_time))

Train time = 0.05
Accuracy = 0.9130
Prediction time = 5.58


Alright, now that we have a simple baseline, let's start working in Theano.  Before we jump to multi-layer neural networks though, let's train a logistic regression model to make certain we're using Theano correctly. 

Recall from Josh's regression lecture the four key components: (1) parameters, (2) model, (3) cost function, and (4) objective. 

Two notes relevant at this point:

First, logistic regression can be thought of as a neural network with no hidden layers. The output values are just the dot product of the inputs and the edge weights.

Second, we have 10 classes. We can either train separate one vs all classifiers using sigmoid activation, which would be a hassle, or we can use the softmax activation, which is essentially a multi-class version of sigmoid. We'll use Theano's built-in implementation of softmax.

We'll use cross-entropy as a cost function.  Cross entropy only considers the error between the true class and the prediction, and not the errors for the false classes.  This tends to cause the network to converge faster.  We'll use Theano's built-in cross entropy function.

The objective is minimize the cost, and to do that we'll use batch gradient descent.

Exercise: What are the differences between batch, stochastic, and mini-batch gradient descent?  What are the implications of each for working on large datasets?

We'll use Theano's built-in gradient function.  Exercise: Do you recall from Josh's lecture what the gradient is for beta in logistic regression?

In [5]:
## Model
model = Sequential() 
model.add(Dense(10, input_dim=784, activation='softmax')) 

## Cost function & Objective (and solver)
sgd = optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels_b, shuffle=False, batch_size=numTrainExamples,verbose=0, epochs=50) 
score = model.evaluate(test_data, test_labels_b, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

('Test score:', 1.869044734954834)
('Test accuracy:', 0.5)


Exercise:  What do you expect to happen if we convert batch gradient descent to stochastic gradient descent?  Why?

Let's try it...

In [6]:
## Model
model = Sequential() 
model.add(Dense(10, input_dim=784, activation='softmax')) 

## Cost function & Objective (and solver)
sgd = optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels_b, shuffle=False, batch_size=1,verbose=0, epochs=50) 
score = model.evaluate(test_data, test_labels_b, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

('Test score:', 0.48422622156143186)
('Test accuracy:', 0.8765)
